## This is my first Project in LLM Engineering.
### Project challenge
- Create a product that build a broucher for a company to be used for prospective clients and potential recruits.
- We will providec ompany name and the website. 

#### We will first import the requirements

In [1]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from scraper import    fetch_website_links , fetch_website_contents
from IPython.display import display, Markdown, update_display
import gradio as gr


d:\UDEMY\LLM-engineering-master-ai\Week1\AI-website-brochure-generator\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Initialise the API key

In [2]:
load_dotenv()
api_key=os.getenv("OpenAI_API_KEY")
if api_key and len(api_key)>10:
    print("key loaded successfully")
else:
    print("Unable to load api key.")
MODEL= 'gpt-5-nano'
openai=OpenAI()

key loaded successfully


#### Fetch website link

In [9]:
links=fetch_website_links("https://huggingface.com")
links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/zai-org/GLM-Image',
 '/Lightricks/LTX-2',
 '/fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA',
 '/openbmb/AgentCPM-Explore',
 '/google/translategemma-4b-it',
 '/models',
 '/spaces/multimodalart/qwen-image-multiple-angles-3d-camera',
 '/spaces/mrfakename/Z-Image-Turbo',
 '/spaces/Wan-AI/Wan2.2-Animate',
 '/spaces/prithivMLmods/Qwen-Image-Edit-2511-LoRAs-Fast',
 '/spaces/r3gm/wan2-2-fp8da-aoti-preview',
 '/spaces',
 '/datasets/MiniMaxAI/OctoCodingBench',
 '/datasets/HuggingFaceFW/finetranslations',
 '/datasets/Alibaba-Apsara/Superior-Reasoning-SFT-gpt-oss-120b',
 '/datasets/miromind-ai/MiroVerse-v0.1',
 '/datasets/nvidia/embed-nemotron-dataset-v1',
 '/datasets',
 '/join',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/inference/models',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '

#### The above code will produce many irrelvent links from the website. So in order to find the relevent links, we will use the chatgpt. 
**This can be achieved through below steps**
- We use the gpt-5-nano to read the links on webpage and then respond in structured JSON
- We will use the **One shot prompting** for this.


In [3]:

system_prompt="""
- You are provied with a list of the links found on a webpage.
- You are able to decide which of the links would be most relevant to include in a brochure about the company, such
as links to about us page, or a career/job page.
- You should respond in JSON format as shown in below example.
{
    "Links":[{"type":"about page", "url":"https://full.url/about"}]
}
"""

*Creating a function so that it can take the list of the links and then pass it to the prompt input*

In [4]:
def list_of_urls(url):
    user_prompt= f"""
    Here is the list of links on the websites {url}
    You need to decide which of these are relevent to creater company brochure, and you respond with full https URL in JSON format.
    Dont include Terms of service, T&C, Privacy, email links.

"""
    links=fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [12]:
print(list_of_urls("https://huggingface.com"))


    Here is the list of links on the websites https://huggingface.com
    You need to decide which of these are relevent to creater company brochure, and you respond with full https URL in JSON format.
    Dont include Terms of service, T&C, Privacy, email links.

/
/models
/datasets
/spaces
/docs
/enterprise
/pricing
/login
/join
/spaces
/models
/zai-org/GLM-Image
/Lightricks/LTX-2
/fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA
/openbmb/AgentCPM-Explore
/google/translategemma-4b-it
/models
/spaces/multimodalart/qwen-image-multiple-angles-3d-camera
/spaces/mrfakename/Z-Image-Turbo
/spaces/Wan-AI/Wan2.2-Animate
/spaces/prithivMLmods/Qwen-Image-Edit-2511-LoRAs-Fast
/spaces/r3gm/wan2-2-fp8da-aoti-preview
/spaces
/datasets/MiniMaxAI/OctoCodingBench
/datasets/HuggingFaceFW/finetranslations
/datasets/Alibaba-Apsara/Superior-Reasoning-SFT-gpt-oss-120b
/datasets/miromind-ai/MiroVerse-v0.1
/datasets/nvidia/embed-nemotron-dataset-v1
/datasets
/join
/enterprise
/enterprise
/enterprise
/enterpris

In [5]:
def select_relevent_links(url):
    response=openai.chat.completions.create(
        model=MODEL, 
        messages=[
            {"role":"system", "content": system_prompt},
            {"role":"user", "content": list_of_urls(url)}
        ],
        response_format={"type":"json_object"}
        )
    result=response.choices[0].message.content
    links=json.loads(result)
    return links

In [14]:
select_relevent_links("https://huggingface.com")

{'Links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'career page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'learn/resources', 'url': 'https://huggingface.co/learn'},
  {'type': 'docs', 'url': 'https://huggingface.co/docs'}]}

*Now we are going to create the company broucher by creating the fucntion that do below things:*

- Fetch the website contents
- Fetching the links from the content

In [6]:
def fetch_page_and_all_relevant_link(url):
    contents= fetch_website_contents(url)
    relevent_links=select_relevent_links(url)
    result=f"\n\n ### Landing Page:\n\n{contents}\n ### Relevant Links:\n"
    for link in relevent_links['Links']:
        result+= f"\n\n### Link: {link['type']}\n"
        result+= fetch_website_contents(link ["url"])
    return result

In [16]:
print(fetch_page_and_all_relevant_link("https://huggingface.com"))



 ### Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-Image
Updated
4 days ago
•
6.64k
•
838
Lightricks/LTX-2
Updated
about 14 hours ago
•
1.54M
•
1.15k
fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA
Updated
12 days ago
•
53.1k
•
753
openbmb/AgentCPM-Explore
Updated
about 17 hours ago
•
1.55k
•
347
google/translategemma-4b-it
Updated
4 days ago
•
12.8k
•
316
Browse 2M+ models
Spaces
Running
on
Zero
Featured
933
Qwen Image Multiple Angles 3D Camera
🎥
933
Adjust camera angles in images using 3D controls or sliders
Running
on
Zero
MCP
1.59k
Z Image Turbo
🖼
1.59k
Generate stunning images from text descriptions in seconds
Running
Featured
4.26k
Wan2.2 Animate
👁
4.26k
Wan2.

#### Now we will write the system prompt for creting the Compant broucher.

In [7]:
System_prompt_broucher="""
- You are an assistant that analyse the contents of several relevnt pages from a company website and
create a short broucher about the company for prospective customers, investors and recruits.
- You respond in markdown without code blocks.
- You inclde details of company culture, customers and career/jobs if you have the information
"""

In [8]:
def get_user_prompt_broucher(company_name, url):
    user_prompt=f"""
- Your are looking at a company called {company_name}
- Here are the contents of its landing page and other relevant pages.
- You can use this information to build a short broucher of the company in MARKDOWN without code blocks. \n\n
"""
    user_prompt+=fetch_page_and_all_relevant_link(url)
    user_prompt= user_prompt[:5000]
    return user_prompt

In [19]:
get_user_prompt_broucher("hugingface", "https://huggingface.com")

'\n- Your are looking at a company called hugingface\n- Here are the contents of its landing page and other relevant pages.\n- You can use this information to build a short broucher of the company in MARKDOWN without code blocks. \n\n\n\n\n ### Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-Image\nUpdated\n4 days ago\n•\n6.64k\n•\n838\nLightricks/LTX-2\nUpdated\nabout 14 hours ago\n•\n1.54M\n•\n1.15k\nfal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA\nUpdated\n12 days ago\n•\n53.1k\n•\n753\nopenbmb/AgentCPM-Explore\nUpdated\nabout 17 hours ago\n•\n1.55k\n•\n347\ngoogle/translategemma-4b-it\nUpdated\n4 days ago\n•\n12.8k\n•\n316\nBrowse 2M+ models\nS

#### Now we are going to create the broucher

In [9]:
def create_broucher(company_name, url):
    response= openai.chat.completions.create(
        model="gpt-4.1-mini", 
        messages=[
        {"role":"system", "content": System_prompt_broucher},
        {"role":"user", "content":get_user_prompt_broucher(company_name, url)}
        ],)
        
    result=response.choices[0].message.content
    display(Markdown(result))
    

In [25]:
create_broucher("huggingFace", "https://huggingface.com")

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face – Building the Future of AI Collaboration

## About Hugging Face
Hugging Face is the leading AI community and platform where machine learning enthusiasts and professionals collaborate to build, share, and deploy state-of-the-art AI models, datasets, and applications. With a collection of over 2 million models and 500,000+ datasets across multiple modalities—including text, image, video, audio, and even 3D—Hugging Face empowers the global machine learning community to innovate and accelerate their projects.

## Platform Highlights

- **Models & Datasets:** Explore and contribute to a massive repository of cutting-edge machine learning models and diverse datasets.
- **Spaces:** Host and engage with AI applications in a seamless environment, supporting interactive demos and innovation sharing.
- **Community First:** A vibrant ecosystem focused on collaboration, learning, and open source innovation.
- **Multimodal Support:** Support for a wide range of AI modalities helps users tackle complex, real-world problems.
- **Open Source Stack:** Built on a fully open-source infrastructure that accelerates experimentation and deployment.

## For Enterprises

Hugging Face delivers an enterprise-grade AI platform designed to scale organizations' machine learning initiatives with advanced features such as:

- Single Sign-On (SSO) for secure identity management
- Granular access control and resource group management
- Comprehensive audit logs for tracking repository activity
- Advanced compute options like ZeroGPU with boosted quotas
- Private dataset viewing and additional private storage
- Analytics dashboard for usage insights
- Flexible billing, including yearly commitment options
- Priority support for dedicated assistance

Starting at $20/user/month for teams, enterprises have access to the world's most advanced AI collaboration platform with enterprise-grade security, compliance, and support.

## Individual & Pro Accounts

For individuals and developers, the Pro subscription at $9/month offers:

- Increased private storage and inference credits
- Higher ZeroGPU quota and priority in compute queue
- Enhanced Spaces hosting and developer tools
- Ability to publish blog articles and showcase projects
- Private dataset viewing capabilities

## Company Culture

Hugging Face fosters a collaborative and inclusive culture emphasizing:

- Open collaboration and knowledge sharing within the AI community
- Encouragement of creativity and innovation in machine learning
- Support for both newcomers and experts to connect and grow
- Dedication to building an ethical and accessible AI ecosystem

## Careers and Opportunities

Hugging Face is actively growing and offers opportunities for talented individuals passionate about AI to join their team. With a focus on impacting the future of machine learning and empowering the community, career openings span research, engineering, product development, and more. Visit their careers page to explore current openings and become part of a forward-thinking, mission-driven company.

---

## Join Hugging Face Today

Whether you're an individual innovator, a researcher, or an organization looking to leverage AI, Hugging Face provides the tools, community, and infrastructure to accelerate your machine learning journey.  

**Explore:** [https://huggingface.co](https://huggingface.co)  
**Sign Up:** Start building and collaborating today.  
**Enterprise Solutions:** Contact sales to scale your AI operations securely and efficiently.

*Hugging Face — the AI community building the future.*

#### If we want to get the respnse in streamed way, we can do this to as demonstrated below.

In [12]:
def stream_broucher(company_name, url):
    stream=openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role":"system", "content":System_prompt_broucher},
                  {"role":"system", "content": get_user_prompt_broucher(company_name, url)}], stream=True
    )
    response=""
    display_handle=display(Markdown(""), display_id=True)
    for chunk in stream:
        response+= chunk.choices[0].delta.content or ''
        #update_display(Markdown(response), display_id=display_handle.display_id)
        yield response

In [33]:
stream_broucher("huggingFace", "https://huggingface.com")

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face: The AI Community Building the Future

---

## About Hugging Face

Hugging Face is a leading collaboration platform designed for the global machine learning (ML) community. It serves as the central hub where ML engineers, scientists, and enthusiasts can share, explore, and build open-source machine learning models, datasets, and applications. The company is committed to empowering the next generation of ML professionals by providing state-of-the-art tools, a growing community, and fostering an open and ethical AI future.

### Key Highlights:
- **Open and Collaborative**: Host and experiment with more than 2 million models and over 500,000 datasets across multiple modalities including text, images, video, audio, and 3D.
- **Community-Driven**: A fast-growing community contributes models and datasets, accelerating innovation and knowledge sharing.
- **Innovation at Core**: Home to some of the most widely-used open-source ML libraries and tools with a talented science team constantly advancing AI technology.

---

## Platform Features

- **Models:** Browse and deploy a vast library of pre-trained ML models in diverse categories.
- **Datasets:** Access and share over half a million datasets optimized for various ML tasks.
- **Spaces:** Deploy ML applications directly on the platform with rich interactive demos.
- **Collaboration:** Seamlessly host and co-develop ML projects, building your portfolio and reputation within the AI community.
- **Multi-Modality Support:** Supports building and deploying AI solutions across text, images, videos, audio, and even 3D data.

---

## Enterprise Offerings

Hugging Face provides tailored solutions for teams and enterprises looking to scale their AI work with enhanced security and support:

- **Team Hub:** Starting at $20/user/month, enabling efficient collaboration and resource management.
- **Enterprise Hub:** Flexible contracts with enterprise-grade features including Single Sign-On (SSO), granular access control, audit logs, data region selection, centralized token management, and dedicated support.
- **Advanced Compute & Storage:** Additional compute resources such as ZeroGPU quota boosts and private storage options.
- **Data Privacy:** Private datasets viewer and secure, governed environments ideal for sensitive workloads.
- **Analytics:** Comprehensive dashboards to monitor repository usage and team performance.

---

## Company Culture

- **Community-Focused:** Hugging Face thrives on openness and collaboration, encouraging inclusive participation to accelerate AI advancements.
- **Innovative & Ethical AI:** The company is strongly committed to building an ethical AI future through transparency, sharing, and leveraging cutting-edge technologies.
- **Talent & Growth:** Hosting a vibrant community of top AI researchers and practitioners, Hugging Face is at the forefront of driving the AI revolution and nurturing talent.

---

## Careers & Opportunities

- Hugging Face is actively growing and offers various roles for those passionate about machine learning and AI.
- The company values diverse skills spanning research, engineering, product, and community building.
- Prospective candidates can join to contribute to open-source projects, be part of an innovative team, and work at the cutting edge of AI technology.

Explore current openings on their careers page to become part of the AI community shaping the future.

---

## Connect & Explore

- Visit [huggingface.co](https://huggingface.co) to browse models, datasets, and AI applications.
- Join the active community discussions on GitHub, Discord, LinkedIn, and Twitter.
- Build, share, and collaborate with the world’s fastest-growing AI platform.

---

Hugging Face is where the future of AI is being built—join the revolution today!

In [ ]:
company_name=gr.Textbox(label="Company_name")
Company_url=gr.Textbox(label="Company URL")
output_text= gr.Markdown(label="Response")

view= gr.Interface(
    fn=stream_broucher,
    title="Create Company Broucher",
    inputs=[company_name, Company_url],
    outputs=[output_text],
    examples=[
      ["huggingFace", "https://huggingface.com"]   
    ],
    flagging_mode="never"

)
view.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7883
* To create a public link, set `share=True` in `launch()`.


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the leading AI community and collaboration platform dedicated to building the future of machine learning. It serves as a central hub where ML engineers, data scientists, researchers, and enthusiasts from around the world converge to create, share, discover, and experiment with open-source machine learning models, datasets, and applications.

---

## What We Offer

- **Extensive Model & Dataset Repository**
  - Browse over 2 million machine learning models covering text, image, video, audio, and even 3D modalities.
  - Access a vast collection of 500,000+ datasets to power your AI research and projects.

- **Hugging Face Spaces**
  - A showcase and hosting service for over 1 million interactive AI applications built by the community.
  - Easily deploy and demo your ML applications with zero infrastructure management.

- **Open Source Ecosystem**
  - Utilize the Hugging Face open-source stack for faster model building and deployment.
  - Collaborate and contribute to a rapidly expanding ecosystem emphasizing openness, reproducibility, and ethical AI.

- **Enterprise & Compute Services**
  - For businesses requiring scalable and secure solutions, Hugging Face offers paid compute resources and enterprise-grade services that accelerate AI adoption.

---

## Community & Culture

- **Collaborative Innovation**
  - At its core, Hugging Face fosters a vibrant, fast-growing global community where knowledge sharing and collaboration are paramount.
  - Public hosting of unlimited models, datasets, and applications enables transparency and collective progress.

- **Ethical AI Commitment**
  - Hugging Face empowers the next generation of machine learning talent with tools and a platform grounded in openness and ethical AI principles.
  
- **Inclusion & Accessibility**
  - The platform welcomes users of all skill levels—from students to seasoned AI experts—to build portfolios, share work, and grow professionally.

---

## Who Uses Hugging Face?

- Machine learning engineers and researchers seeking state-of-the-art models and datasets.
- AI startups and enterprises leveraging enterprise services for large-scale deployments.
- Educational institutions and students building ML skills and portfolios.
- Developers and creators building innovative AI-driven applications across industries.

---

## Career Opportunities

- Hugging Face is continuously growing and invites talented individuals passionate about open-source machine learning to join the team.
- Roles typically span AI research, software engineering, community engagement, and enterprise support.
- Expect to work in a collaborative, mission-driven environment focused on making AI accessible, open, and ethical.

---

## Get Involved & Learn More

- **Explore Models, Datasets, and Applications:** Join the community at [huggingface.co](https://huggingface.co).
- **Sign up and start collaborating:** Build your machine learning portfolio and accelerate your projects.
- **Enterprise Solutions:** Contact Hugging Face for scalable AI deployments tailored to your business needs.

---

### Brand Highlights

- Recognizable by its vibrant color palette including yellow (#FFD21E) and orange (#FF9D00).
- Utilized globally in AI research, development, and applications through open-source commitment.

---

**Hugging Face** – *The AI community building the future.*  
Join us to collaborate, innovate, and share the journey toward open and ethical AI for all.